# Clean up Terra data sets
remove old data set. Typically these sets contained failed sample that needed to be re reun
```
Andrew E. Davidson
aedavids@ucsc.edu
```

In [1]:
import pandas as pd
from pandas import DataFrame

In [2]:
root = "terraDataModels-hack/test-aedavids-proj/TCGA"
rootDir = "../../../terraDataModels/test-aedavids-proj/TCGA"
listOfWorkSpacePath = rootDir + "/" + "listOfWorkSpaces.txt" 
workspaceDF = pd.read_csv( listOfWorkSpacePath )
workSpaceNamesList = workspaceDF.loc[:, "wokspace"].to_list()
workSpaceNamesList

['TCGA_ACC_ControlledAccess_V1-0_DATA_edu_ucsc_kim_lab',
 'TCGA_BLCA_ControlledAccess_V1-0_DATA_edu_ucsc_kim_lab',
 'TCGA_BRCA_ControlledAccess_V1-0_DATA_edu_ucsc_kim_lab',
 'TCGA_CESC_ControlledAccess_V1-0_DATA_edu_ucsc_kim_lab',
 'TCGA_CHOL_ControlledAccess_V1-0_DATA_edu_ucsc_kim_lab',
 'TCGA_COAD_ControlledAccess_V1-0_DATA_edu_ucsc_kim_lab',
 'TCGA_DLBC_ControlledAccess_V1-0_DATA _edu_ucsc_kim_lab',
 'TCGA_ESCA_ControlledAccess_V1-0_DATA_edu_ucsc_kim_lab',
 'TCGA_GBM_ControlledAccess_V1-0_DATA_edu_ucsc_kim_lab',
 'TCGA_HNSC_ControlledAccess_V1-0_DATA_edu_ucsc_kim_lab',
 'TCGA_KICH_ControlledAccess_V1-0_DATA_edu_ucsc_kim_lab',
 'TCGA_KIRC_ControlledAccess_V1-0_DATA_edu_ucsc_kim_lab',
 'TCGA_KIRP_ControlledAccess_V1-0_DATA_edu_ucsc_kim_lab',
 'TCGA_LAML_ControlledAccess_V1-0_DATA_edu_ucsc_kim_lab',
 'TCGA_LGG_ControlledAccess_V1-0_DATA_edu_ucsc_kim_lab',
 'TCGA_LIHC_ControlledAccess_V1-0_DATA_edu_ucsc_kim_lab',
 'TCGA_LUAD_ControlledAccess_V1-0_DATA_edu_ucsc_kim_lab',
 'TCGA_LUAD_Cont

In [3]:
# upload sample_set_entity.tsv  and sample_set_membership.tsv failed. http timeout
# think it may be because they are really big. We ran createFailedSamplesData.set.ipybn many 
# time while debugging
failedToUploadList = ["TCGA_BRCA_ControlledAccess_V1-0_DATA_edu_ucsc_kim_lab",
                      "TCGA_LUAD_ControlledAccess_V1-0_DATA_edu_ucsc_kim_lab", 
                      "TCGA_LUAD_ControlledAccess_V1-0_DATA_edu_ucsc_kim_lab"]

In [4]:
def readDataModel( rootDir, workspaceName, entityName ) :
    dataModelTSV = rootDir + "/" + workspaceName + "/" + entityName + ".tsv"
    dataModelDF = pd.read_csv(dataModelTSV, delimiter='\t')
    return dataModelDF

In [5]:
def saveDataModel( rootDir, workspaceName, entityName, dataModelDF ) :
    dataModelTSV = rootDir + "/" + workspaceName + "/" + entityName + ".tsv"
    print("writing {}".format(dataModelTSV))
    dataModelDF.to_csv(dataModelTSV, sep='\t', index=False)

In [6]:
def deleteSets( workspaceName, setList):
    sampleSetEntityDF = readDataModel( rootDir, workspaceName, "sample_set_entity" )
    sampleSetMembershipDF = readDataModel( rootDir, workspaceName, "sample_set_membership" )
    
    print("{} starting membership".format(workspaceName))
    print(sampleSetMembershipDF.groupby('membership:sample_set_id').count())
    print()
    
    for s in setList:
        deleteMembersPS = sampleSetMembershipDF['membership:sample_set_id'] == s
        memberRowsPS =  sampleSetMembershipDF['membership:sample_set_id'] != s
        sampleSetMembershipDF = sampleSetMembershipDF.loc[memberRowsPS, :]
        print("removing set '{} with items {}".format(s, sum( deleteMembersPS) ))  
        
        entityRowsPS = sampleSetEntityDF['entity:sample_set_id'] != s
        sampleSetEntityDF = sampleSetEntityDF.loc[entityRowsPS, :]
        
    return (sampleSetEntityDF, sampleSetMembershipDF)

rmSets = ["salmonSingleEndReadTask-wdl_2021-10-06T22-53-16"]
cleanSampleSetEntityDF, cleanSampleSetMembershipDF \
    = deleteSets( "TCGA_BRCA_ControlledAccess_V1-0_DATA_edu_ucsc_kim_lab", rmSets )

TCGA_BRCA_ControlledAccess_V1-0_DATA_edu_ucsc_kim_lab starting membership
                                                 sample
membership:sample_set_id                               
BRCA                                               2263
failedPair-2022-02-17-15-56                          62
salmonSingleEndReadTask-wdl_2021-10-06T22-53-16    2263

removing set 'salmonSingleEndReadTask-wdl_2021-10-06T22-53-16 with items 2263


In [7]:
cleanSampleSetEntityDF

,entity:sample_set_id
0,BRCA
2,failedPair-2022-02-17-15-56


In [8]:
cleanSampleSetMembershipDF.groupby('membership:sample_set_id').count()

,sample
membership:sample_set_id,
BRCA,2263
failedPair-2022-02-17-15-56,62


In [9]:
saveDataModel(rootDir, "TCGA_BRCA_ControlledAccess_V1-0_DATA_edu_ucsc_kim_lab", "sample_set_entity",   
              cleanSampleSetEntityDF)
saveDataModel(rootDir, "TCGA_BRCA_ControlledAccess_V1-0_DATA_edu_ucsc_kim_lab", "sample_set_membership",
              cleanSampleSetMembershipDF)

writing ../../../terraDataModels/test-aedavids-proj/TCGA/TCGA_BRCA_ControlledAccess_V1-0_DATA_edu_ucsc_kim_lab/sample_set_entity.tsv
writing ../../../terraDataModels/test-aedavids-proj/TCGA/TCGA_BRCA_ControlledAccess_V1-0_DATA_edu_ucsc_kim_lab/sample_set_membership.tsv
